In [1]:
import pandas as pd
import numpy
import time
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

In [2]:
import pandas as pd
import numpy
df=pd.read_csv("交易資料集.csv").dropna()
df

,ITEM_ID,ITEM_NO,PRODUCT_TYPE,CUST_ID,TRX_DATE,INVOICE_NO,QUANTITY
0,3217532,M25P40-VMN6TPB,MEMORY_EMBEDED,3218,2016/7/26,CX47348203,2500
1,3326781,AU80610006237AASLBX9,CPU / MPU,2470,2016/7/11,CX47346522,50
2,740487,MMBD2837LT1G,DISCRETE,16135,2016/7/27,CX47348534,3000
3,3434776,IHLP1616ABER2R2M11,PEMCO,999999999,2016/7/29,A20160700174,0
4,70072,MMBT3906LT1G,DISCRETE,2356,2016/7/6,CX47346184,12000
...,...,...,...,...,...,...,...
157391,133619,"74LVC2G17GW,125",LOGIC IC,2579,2016/7/11,216072287,1644
157392,15231095,MT41K256M16TW-107:P TRAY,MEMORY_EMBEDED,43262,2016/7/28,216077517,300000
157393,14864010,MT36KSF2G72PZ-1G6E1FE,MEMORY_EMBEDED,47302,2016/7/7,A10123295,-1
157394,15220039,CM8066201920103SR2L2,CPU / MPU,49263,2016/7/22,A10123946,-2


In [3]:
df=df[df.QUANTITY > 0]
df=df[df.PRODUCT_TYPE != 'Null']
columns = ['ITEM_ID','ITEM_NO','CUST_ID', 'TRX_DATE','QUANTITY']
df=df.drop(columns,axis=1)
df

,PRODUCT_TYPE,INVOICE_NO
0,MEMORY_EMBEDED,CX47348203
1,CPU / MPU,CX47346522
2,DISCRETE,CX47348534
4,DISCRETE,CX47346184
6,LINEAR IC,CX47347899
...,...,...
157389,LOGIC IC,216071271
157390,LOGIC IC,216075314
157391,LOGIC IC,216072287
157392,MEMORY_EMBEDED,216077517


In [4]:
g = df.groupby("INVOICE_NO")
g = g.groups

INVOICE_NO_dataset = []
PRODUCT_TYPE_dataset = []
for i in g:
    item = []
    for j in g.get(i):
        item.append(df.at[j, "PRODUCT_TYPE"])
    INVOICE_NO_dataset.append(i)
    PRODUCT_TYPE_dataset.append(item)

In [5]:
data = {
    "INVOICE_NO_dataset": INVOICE_NO_dataset,
    "PRODUCT_TYPE_dataset": PRODUCT_TYPE_dataset
}
data = pd.DataFrame(data)

te = TransactionEncoder()
te_items = te.fit(PRODUCT_TYPE_dataset).transform(PRODUCT_TYPE_dataset)
te_items = pd.DataFrame(te_items, columns=te.columns_)

In [6]:
#Apriori1
start= time.perf_counter()
apr1 = apriori(te_items, min_support=0.002,use_colnames=True)
end = time.perf_counter()
print("執行時間:", end-start, "秒")

apr1 = association_rules(apr1, metric="lift", min_threshold=1)
apr1

執行時間: 0.031238399999999444 秒


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(CPU / MPU),(CHIPSET / ASP),0.110746,0.044112,0.005343,0.048243,1.093658,0.000458,1.004341
1,(CHIPSET / ASP),(CPU / MPU),0.044112,0.110746,0.005343,0.121118,1.093658,0.000458,1.011802
2,(LINEAR IC),(LOGIC IC),0.260398,0.121541,0.043454,0.166877,1.373012,0.011805,1.054417
3,(LOGIC IC),(LINEAR IC),0.121541,0.260398,0.043454,0.357529,1.373012,0.011805,1.151184
4,(MEMORY_EMBEDED),(MEMORY_SYSTEM),0.066716,0.021234,0.002055,0.030801,1.450540,0.000638,1.009871
...,...,...,...,...,...,...,...,...,...
91,"(LINEAR IC, LOGIC IC)","(OTHERS, DISCRETE)",0.043454,0.006658,0.002219,0.051072,7.670870,0.001930,1.046804
92,(DISCRETE),"(OTHERS, LINEAR IC, LOGIC IC)",0.257905,0.002849,0.002219,0.008605,3.019901,0.001484,1.005806
93,(OTHERS),"(DISCRETE, LINEAR IC, LOGIC IC)",0.113403,0.014713,0.002219,0.019570,1.330100,0.000551,1.004954
94,(LINEAR IC),"(OTHERS, DISCRETE, LOGIC IC)",0.260398,0.002767,0.002219,0.008523,3.079827,0.001499,1.005805


In [7]:
#Apriori2
start= time.perf_counter()
apr2 = apriori(te_items, min_support=0.005,use_colnames=True)
end = time.perf_counter()
print("執行時間:", end-start, "秒")

apr2 = association_rules(apr2, metric="lift", min_threshold=1)
apr2

執行時間: 0.019381400000000326 秒


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(CPU / MPU),(CHIPSET / ASP),0.110746,0.044112,0.005343,0.048243,1.093658,0.000458,1.004341
1,(CHIPSET / ASP),(CPU / MPU),0.044112,0.110746,0.005343,0.121118,1.093658,0.000458,1.011802
2,(LINEAR IC),(LOGIC IC),0.260398,0.121541,0.043454,0.166877,1.373012,0.011805,1.054417
3,(LOGIC IC),(LINEAR IC),0.121541,0.260398,0.043454,0.357529,1.373012,0.011805,1.151184
4,"(DISCRETE, LINEAR IC)",(LOGIC IC),0.034906,0.121541,0.014713,0.421507,3.468026,0.010471,1.518530
5,"(DISCRETE, LOGIC IC)",(LINEAR IC),0.026988,0.260398,0.014713,0.545178,2.093634,0.007686,1.626134
6,"(LINEAR IC, LOGIC IC)",(DISCRETE),0.043454,0.257905,0.014713,0.338588,1.312841,0.003506,1.121986
7,(DISCRETE),"(LINEAR IC, LOGIC IC)",0.257905,0.043454,0.014713,0.057049,1.312841,0.003506,1.014417
8,(LINEAR IC),"(DISCRETE, LOGIC IC)",0.260398,0.026988,0.014713,0.056503,2.093634,0.007686,1.031282
9,(LOGIC IC),"(DISCRETE, LINEAR IC)",0.121541,0.034906,0.014713,0.121055,3.468026,0.010471,1.098014


In [8]:
#Apriori3
start= time.perf_counter()
apr3 = apriori(te_items, min_support=0.008,use_colnames=True)
end = time.perf_counter()
print("執行時間:", end-start, "秒")

apr3 = association_rules(apr3, metric="lift", min_threshold=1)
apr3

執行時間: 0.013638700000000448 秒


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(LINEAR IC),(LOGIC IC),0.260398,0.121541,0.043454,0.166877,1.373012,0.011805,1.054417
1,(LOGIC IC),(LINEAR IC),0.121541,0.260398,0.043454,0.357529,1.373012,0.011805,1.151184
2,"(DISCRETE, LINEAR IC)",(LOGIC IC),0.034906,0.121541,0.014713,0.421507,3.468026,0.010471,1.518530
3,"(DISCRETE, LOGIC IC)",(LINEAR IC),0.026988,0.260398,0.014713,0.545178,2.093634,0.007686,1.626134
4,"(LINEAR IC, LOGIC IC)",(DISCRETE),0.043454,0.257905,0.014713,0.338588,1.312841,0.003506,1.121986
5,(DISCRETE),"(LINEAR IC, LOGIC IC)",0.257905,0.043454,0.014713,0.057049,1.312841,0.003506,1.014417
6,(LINEAR IC),"(DISCRETE, LOGIC IC)",0.260398,0.026988,0.014713,0.056503,2.093634,0.007686,1.031282
7,(LOGIC IC),"(DISCRETE, LINEAR IC)",0.121541,0.034906,0.014713,0.121055,3.468026,0.010471,1.098014


In [9]:
#FP-Growth support = 0.002
start = time.perf_counter()
result = fpgrowth(te_items, min_support=0.002, use_colnames=True)
end = time.perf_counter()
print("執行時間:", end-start, "秒")

fp1 = association_rules(result, metric="lift", min_threshold=1)
fp1

執行時間: 0.2372955999999995 秒


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(OTHERS, LINEAR IC)",(LOGIC IC),0.005233,0.121541,0.002849,0.544503,4.479995,0.002213,1.928571
1,"(OTHERS, LOGIC IC)",(LINEAR IC),0.003754,0.260398,0.002849,0.759124,2.915247,0.001872,3.070470
2,(LINEAR IC),"(OTHERS, LOGIC IC)",0.260398,0.003754,0.002849,0.010943,2.915247,0.001872,1.007269
3,(LOGIC IC),"(OTHERS, LINEAR IC)",0.121541,0.005233,0.002849,0.023445,4.479995,0.002213,1.018649
4,"(OTHERS, DISCRETE)",(LOGIC IC),0.006658,0.121541,0.002767,0.415638,3.419736,0.001958,1.503279
...,...,...,...,...,...,...,...,...,...
91,(OPTICAL AND SENSOR),"(DISCRETE, LOGIC IC)",0.039865,0.026988,0.002192,0.054983,2.037323,0.001116,1.029624
92,(DISCRETE),"(OPTICAL AND SENSOR, LOGIC IC)",0.257905,0.002438,0.002192,0.008499,3.485307,0.001563,1.006112
93,(LOGIC IC),"(OPTICAL AND SENSOR, DISCRETE)",0.121541,0.005562,0.002192,0.018034,3.242436,0.001516,1.012701
94,(CPU / MPU),(CHIPSET / ASP),0.110746,0.044112,0.005343,0.048243,1.093658,0.000458,1.004341


In [10]:
#FP-Growth support = 0.005
start = time.perf_counter()
result = fpgrowth(te_items, min_support=0.005, use_colnames=True)
end = time.perf_counter()
print("執行時間:", end-start, "秒")

fp2 = association_rules(result, metric="lift", min_threshold=1)
fp2

執行時間: 0.19001439999999992 秒


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(LINEAR IC),(LOGIC IC),0.260398,0.121541,0.043454,0.166877,1.373012,0.011805,1.054417
1,(LOGIC IC),(LINEAR IC),0.121541,0.260398,0.043454,0.357529,1.373012,0.011805,1.151184
2,"(DISCRETE, LINEAR IC)",(LOGIC IC),0.034906,0.121541,0.014713,0.421507,3.468026,0.010471,1.518530
3,"(DISCRETE, LOGIC IC)",(LINEAR IC),0.026988,0.260398,0.014713,0.545178,2.093634,0.007686,1.626134
4,"(LINEAR IC, LOGIC IC)",(DISCRETE),0.043454,0.257905,0.014713,0.338588,1.312841,0.003506,1.121986
5,(DISCRETE),"(LINEAR IC, LOGIC IC)",0.257905,0.043454,0.014713,0.057049,1.312841,0.003506,1.014417
6,(LINEAR IC),"(DISCRETE, LOGIC IC)",0.260398,0.026988,0.014713,0.056503,2.093634,0.007686,1.031282
7,(LOGIC IC),"(DISCRETE, LINEAR IC)",0.121541,0.034906,0.014713,0.121055,3.468026,0.010471,1.098014
8,(CPU / MPU),(CHIPSET / ASP),0.110746,0.044112,0.005343,0.048243,1.093658,0.000458,1.004341
9,(CHIPSET / ASP),(CPU / MPU),0.044112,0.110746,0.005343,0.121118,1.093658,0.000458,1.011802


In [11]:
#FP-Growth support = 0.008
start = time.perf_counter()
result = fpgrowth(te_items, min_support=0.008, use_colnames=True)
end = time.perf_counter()
print("執行時間:", end-start, "秒")

fp3 = association_rules(result, metric="lift", min_threshold=1)
fp3

執行時間: 0.18430569999999946 秒


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(LINEAR IC),(LOGIC IC),0.260398,0.121541,0.043454,0.166877,1.373012,0.011805,1.054417
1,(LOGIC IC),(LINEAR IC),0.121541,0.260398,0.043454,0.357529,1.373012,0.011805,1.151184
2,"(DISCRETE, LINEAR IC)",(LOGIC IC),0.034906,0.121541,0.014713,0.421507,3.468026,0.010471,1.518530
3,"(DISCRETE, LOGIC IC)",(LINEAR IC),0.026988,0.260398,0.014713,0.545178,2.093634,0.007686,1.626134
4,"(LINEAR IC, LOGIC IC)",(DISCRETE),0.043454,0.257905,0.014713,0.338588,1.312841,0.003506,1.121986
5,(DISCRETE),"(LINEAR IC, LOGIC IC)",0.257905,0.043454,0.014713,0.057049,1.312841,0.003506,1.014417
6,(LINEAR IC),"(DISCRETE, LOGIC IC)",0.260398,0.026988,0.014713,0.056503,2.093634,0.007686,1.031282
7,(LOGIC IC),"(DISCRETE, LINEAR IC)",0.121541,0.034906,0.014713,0.121055,3.468026,0.010471,1.098014


In [12]:
#依據不同參數儲存檔案
apr1[["antecedents", "consequents",'antecedent support','consequent support','support','confidence','lift'
     ,'leverage','conviction']].to_csv("apr1_0.002.csv")
apr2[["antecedents", "consequents",'antecedent support','consequent support','support','confidence','lift'
     ,'leverage','conviction']].to_csv("apr1_0.005.csv")
apr3[["antecedents", "consequents",'antecedent support','consequent support','support','confidence','lift'
     ,'leverage','conviction']].to_csv("apr1_0.008.csv")
fp1[["antecedents", "consequents",'antecedent support','consequent support','support','confidence','lift'
     ,'leverage','conviction']].to_csv("fp1_0.002.csv")
fp2[["antecedents", "consequents",'antecedent support','consequent support','support','confidence','lift'
     ,'leverage','conviction']].to_csv("fp2_0.005.csv")
fp3[["antecedents", "consequents",'antecedent support','consequent support','support','confidence','lift'
     ,'leverage','conviction']].to_csv("fp31_0.008.csv")

In [13]:
print("以下為產品清單")
print("=======================================================================")
print(" (CHIPSET / ASP) (CPU / MPU) (DISCRETE) (LINEAR IC) (LOGIC IC) \n(MEMORY_EMBEDED) (MEMORY_SYSTEM) (OPTICAL AND SENSOR) (OTHERS) (PEMCO)")
print("=======================================================================")
select = input("請輸入產品名稱:(如需查詢多項產品請以逗號隔開，例如:CPU / MPU,DISCRETE)\n")
print("=======================================================================")
print("minsupport共有三種可供選擇，分別為0.002,0.005,0.008")
select_min_support = input("請輸入您要查詢的參數:\n")
if select_min_support == '0.002':
    antecedents = fp1["antecedents"]
    consequents = fp1["consequents"]
if select_min_support == '0.005':
    antecedents = fp2["antecedents"]
    consequents = fp2["consequents"]
if select_min_support == '0.008' :
    antecedents = fp3["antecedents"]
    consequents = fp3["consequents"]

select = [n for n in select.split(",")]
select.sort()
recommend_list = []
index = -1
for i in antecedents:
    index += 1
    f = True
    data = list(i)
    
    for item in select:
        if item not in data:
            f = False
    if not f:
        continue
            
    for recommend in list(consequents[index]):
        recommend_list.append(recommend)   
print("=======================================================================")
if recommend_list:
    print("依據關聯規則向您推薦", recommend_list)
else:
    print("無關聯規則")

以下為產品清單
 (CHIPSET / ASP) (CPU / MPU) (DISCRETE) (LINEAR IC) (LOGIC IC) 
(MEMORY_EMBEDED) (MEMORY_SYSTEM) (OPTICAL AND SENSOR) (OTHERS) (PEMCO)
請輸入產品名稱:(如需查詢多項產品請以逗號隔開，例如:CPU / MPU,DISCRETE)
CPU / MPU,DISCRETE
minsupport共有三種可供選擇，分別為0.002,0.005,0.008
請輸入您要查詢的參數:
0.002
依據關聯規則向您推薦 ['LINEAR IC', 'LOGIC IC']
